In [ ]:
!pip install -U openai-whisper
!apt-get install ffmpeg
!python -m spacy download en_core_web_sm
!apt-get install fonts-noto
!pip install -U openai-whisper
!pip install fpdf
!pip install transformers torch

  Using cached openai-whisper-20240930.tar.gz (800 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tiktoken-0.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cu

In [ ]:
import whisper
import spacy
import re
from datetime import datetime
from transformers import pipeline
from fpdf import FPDF
import os
from google.colab import files

In [ ]:
audio_file_path ="/content/2025-04-01 Council Meeting.mp3"

In [ ]:
whisper_model = whisper.load_model("base")

transcription_result = whisper_model.transcribe(audio_file_path)
transcript_text = transcription_result["text"]

print("🔹 Transcription Complete:\n", transcript_text[:500], "...")


100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 32.8MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🔹 Transcription Complete:
  Thank you very much. This is the RMO Springfield meeting agenda for April 1st, April Fool's Day of 2025. Council meeting is starting at exactly 6 p.m. I'm Mayor Patrick Terrien for the RMO Springfield. I'll introduce Council. As we call this to order, Deputy Mayor Council Award 1 is Glen Fuel. Next will be Council Award 2 is Andy Kaczynski. Next will be Council Award 3 is Council Mark Miller. Next will be Council Award 4 is Melinda Warren. I'll go to land acknowledgments there. The RMO Springfi ...


In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn",device=-1)

transcript_short = " ".join(transcript_text.split()[:700])

print("Transcript Sample:", transcript_short[:500])

summary = summarizer(transcript_short, max_length=2000, min_length=100, do_sample=False)


meeting_summary = summary[0]["summary_text"]
print("Meeting Summary:\n", meeting_summary)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 2000, but your input_length is only 885. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=442)


Transcript Sample: Thank you very much. This is the RMO Springfield meeting agenda for April 1st, April Fool's Day of 2025. Council meeting is starting at exactly 6 p.m. I'm Mayor Patrick Terrien for the RMO Springfield. I'll introduce Council. As we call this to order, Deputy Mayor Council Award 1 is Glen Fuel. Next will be Council Award 2 is Andy Kaczynski. Next will be Council Award 3 is Council Mark Miller. Next will be Council Award 4 is Melinda Warren. I'll go to land acknowledgments there. The RMO Springfie
Meeting Summary:
 RMO Springfield acknowledges that we are gathered on ancestral lands. Most of the residents in the RMO Springfield, of course, get their drinking water from our aquifer. That doesn't say the people who are on pipe water don't because it comes from the same source. I'm hopeful that once the new draft guidelines or the guidelines come into effect on a national level, perhaps the provinces or the health agencies of each province will support those who are in we

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(transcript_text)

date_match = re.search(r"\b(\d{1,2} (?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4})\b", transcript_text)
meeting_date = date_match.group(0) if date_match else datetime.today().strftime("%d %B %Y")

attendees = list(set([ent.text for ent in doc.ents if ent.label_ == "PERSON"]))
attendee_names = ", ".join(attendees) if attendees else "Not Specified"

print(f"Date: {meeting_date}")
print(f"Attendees: {attendee_names}")


Date: 03 April 2025
Attendees: Glenn, john q, Nyland, Pierre Paulier, Kazinsky, millinda glenda, Terran, Lilland, Councillor Taren, Patrick Terrien, Councilor Kaczynski, Y'all, Warren, Brock, Miller, Robert, Christie, a. Grader, Melinda, malinda, Councilor Warren, Warren W. Mayor Fuel, Glen Fuel, Dene Peoples, Councillor Kaczynski, Councilor, Terry, Vienstra, Patrick, Stan Tech, Lelond, Fuel, Janet Na, Councillor Warren, Perbank, Springfield, glen samard, Paul McDonald, Lee, fiel, Janet Nylon, Pierre Polly, Fulge, Kriogi Kri, Richard, Councilor Fuehl, miller, Banc, LaLond, Dawson, Pat Burbank, Levy, Councillor Miller, Janet, Shake Councilor, Melinda Warren, melinda glenda, Councillor Fuehl, Councillor Kuzinski, Nylon, Sharon Lilland, Mark, mayor, Lalonde, mark miller, Andy Kaczynski, springfield levy, Miss Nyland, Kaczynski, Colleen, Garvin Road, Mark Miller, Andy, Bozer, andy kizinski, donna, Councilor Miller, melinda, Leslie Thompson, Zoom, Randy, glendon Melinda


In [ ]:
from transformers import pipeline

extractor = pipeline("text2text-generation", model="google/flan-t5-large", device=-1)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def extract_section(prompt, transcript):

    result = extractor(f"{prompt}\n\n{transcript}", max_length=2000, do_sample=False)
    return result[0]['generated_text'].strip()


action_items = extract_section("Extract the action items and responsibilities from the meeting:", transcript_short)
next_steps = extract_section("List the next steps and follow-up actions from the meeting:", transcript_short)
closing_remarks = extract_section("Summarize the closing remarks from the meeting:", transcript_short)
meeting_time=""

print("Action Items:", action_items)
print("Next Steps:", next_steps)
print("Closing Remarks:", closing_remarks)


Token indices sequence length is longer than the specified maximum sequence length for this model (972 > 512). Running this sequence through the model will result in indexing errors


Action Items: Councillor Miller.
Next Steps: Councillor Miller.
Closing Remarks: The RMO Springfield Council meeting is April 1st, April Fool's Day of 2025. The agenda is as follows: Deputy Mayor Council Award 1 is Glen Fuel. Next will be Council Award 2 is Andy Kaczynski. Next will be Council Award 3 is Mark Miller. Next will be Council Award 4 is Melinda Warren.


In [ ]:
from fpdf import FPDF
import re
from datetime import datetime

class MeetingMinutesPDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 16)
        self.cell(0, 10, "MEETING MINUTES", ln=True, align="C")
        self.ln(5)
        self.add_page_border()

    def footer(self):
        self.set_y(-15)
        self.set_font("Arial", "I", 10)
        self.cell(0, 10, f"Page {self.page_no()}", align="C")

    def add_page_border(self):
        self.rect(5.0, 5.0, 200.0, 287.0)

    def add_section(self, title, content):
        self.set_font("Arial", "B", 12)
        self.cell(0, 8, title, ln=True, align="L", border="B")
        self.ln(3)
        self.set_font("Arial", size=11)
        self.multi_cell(0, 7, content)
        self.ln(5)

    def add_bullet_points(self, title, items):
        self.set_font("Arial", "B", 12)
        self.cell(0, 8, title, ln=True, align="L", border="B")
        self.ln(3)
        self.set_font("Arial", size=11)


        if len(items) > 10:
            displayed_items = items[:10]
            more_count = len(items) - 10
            displayed_items.append(f"+{more_count} more attendees")
        else:
            displayed_items = items

        if displayed_items:
            for item in displayed_items:
                self.cell(5)
                self.cell(0, 7, f"- {item.strip()}", ln=True)
        else:
            self.cell(0, 7, "No attendees listed.", ln=True)

        self.ln(5)

def clean_text(text):
    return re.sub(r'[^\x00-\x7F]+', '', text)

def format_attendees(attendees):
    return [att.strip() for att in attendees.split(",")]

def save_meeting_minutes_to_pdf(date, time, attendees, summary, action_items, next_steps, closing_remarks, filename="Meeting_Minutes.pdf"):
    pdf = MeetingMinutesPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    if not time or time.strip() == "":
        time = datetime.now().strftime("%I:%M %p")


    pdf.add_page()


    pdf.add_section("Date", date)
    pdf.add_section("Time", time)
    pdf.add_bullet_points("Attendees", format_attendees(attendees))


    pdf.add_section("Meeting Overview", clean_text(summary[0]["summary_text"]))
    pdf.add_section("Action Items & Responsibilities", clean_text(action_items) if action_items else "No action items recorded.")
    pdf.add_section("Next Steps & Follow-ups", clean_text(next_steps) if next_steps else "No follow-ups mentioned.")
    pdf.add_section("Closing Remarks", clean_text(closing_remarks) if closing_remarks else "No closing remarks recorded.")

    pdf.output(filename)
    print(f"PDF saved as {filename}")

save_meeting_minutes_to_pdf(meeting_date, meeting_time, attendee_names, summary, action_items, next_steps, closing_remarks)


PDF saved as Meeting_Minutes.pdf


In [ ]:

files.download("Meeting_Minutes.pdf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>